In [2]:
# 通过添加len和getitem方法，来让其可切片

from array import array
import reprlib
import math

class Vector:
    typecode = 'd'
    # _components是受保护的示例属性
    def __init__(self, components) -> None:
        self._components = array(self.typecode, components)

    # 构造迭代器
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self) -> str:
        # 使用reprlib.repr() 函数获取self._components的有限长度表示形式（如array('d',[0.0, 1.0, 2.0, 3.0, 4.0, ...]))
        components = reprlib.repr(self._components)
        # 把字符串插入Vector的构造方法调用之前，去掉前面的array('d'和后面的)。
        components = components[components.find('['): -1]
        return 'Vector({})'.format(components)
    
    def __str__(self) -> str:
        return str(tuple(self))
    
    # 支持切片和索引
    def __len__(self):
        return len(self._components)
    def __getitem__(self, index):
        return self._components[index]
    
    # 直接使用 类型代码 + self._components构建bytes对象。
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + 
                bytes(self._components))
    
    def __eq__(self, other) -> bool:
        return tuple(self) == type(other)

    # 不能使用hypot方法了，因此我们先计算各分量的平方之和，然后再使用sqrt方法开平方。
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [3]:
v1 = Vector([3, 4, 5])
len(v1)

3

In [4]:
v1[0], v1[-1]

(3.0, 5.0)

In [5]:
v7 = Vector(range(7))
v7[1:4]

array('d', [1.0, 2.0, 3.0])

### 10.4.1 切片原理

In [6]:
class MySeq:
    def __getitem__(self, index):
        return index

s = MySeq()

In [7]:
s[1]

1

In [8]:
s[1: 4]

slice(1, 4, None)

In [9]:
s[1: 4: 2]

slice(1, 4, 2)

In [10]:
s[1: 4: 2, 9]

(slice(1, 4, 2), 9)

In [11]:
s[1: 4: 2, 7: 9]

(slice(1, 4, 2), slice(7, 9, None))

In [12]:
slice

slice

In [13]:
dir(slice)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'indices',
 'start',
 'step',
 'stop']

In [14]:
slice(None, 10, 2).indices(5)

(0, 5, 2)

In [15]:
slice(-3, None, None).indices(5)

(2, 5, 1)

### 10.4.2 能处理切片的__getitem__方法

In [16]:
# 通过添加len和getitem方法，来让其可切片

from array import array
import reprlib
import math
import numbers

class Vector:
    typecode = 'd'
    # _components是受保护的示例属性
    def __init__(self, components) -> None:
        self._components = array(self.typecode, components)

    # 构造迭代器
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self) -> str:
        # 使用reprlib.repr() 函数获取self._components的有限长度表示形式（如array('d',[0.0, 1.0, 2.0, 3.0, 4.0, ...]))
        components = reprlib.repr(self._components)
        # 把字符串插入Vector的构造方法调用之前，去掉前面的array('d'和后面的)。
        components = components[components.find('['): -1]
        return 'Vector({})'.format(components)
    
    def __str__(self) -> str:
        return str(tuple(self))
    
    # 支持切片和索引
    def __len__(self):
        return len(self._components)
    def __getitem__(self, index):
        # 获取实例所属的类
        cls = type(self)
        # 如果是slice对象
        if isinstance(index, slice):
            return cls(self._components[index])
        # 如果是其他类型，就返回在这个下标的元素
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))
    
    # 直接使用 类型代码 + self._components构建bytes对象。
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + 
                bytes(self._components))
    
    def __eq__(self, other) -> bool:
        return tuple(self) == type(other)

    # 不能使用hypot方法了，因此我们先计算各分量的平方之和，然后再使用sqrt方法开平方。
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

In [17]:
# 测试上述的getitem方法

v7 = Vector(range(7))

In [18]:
v7[1:4]

Vector([1.0, 2.0, 3.0])

In [19]:
v7[-1]

6.0

In [20]:
v7[1,2]

TypeError: Vector indices must be integers